# Vision Transformer (ViT) - Kaggle Quickstart

This notebook demonstrates how to use the Vision Transformer implementation on Kaggle.

**Steps:**
1. Add this dataset as input to your notebook
2. Run the cells below
3. Start using ViT in your projects!

## Setup

In [1]:
# Import necessary libraries
import sys
import tensorflow as tf
import numpy as np
from tensorflow import keras

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU available: []


In [11]:
# Work in local src directory
# Update this path if your dataset has a different name
sys.path.append('../src')

# Import ViT models
from vit_all_variants import (
    create_vit_s16, create_vit_s32,
    create_vit_b16, create_vit_b32,
    create_vit_l16, create_vit_l32,
    create_vit_h14, create_vit_h16,
    list_variants, compare_variants
)

print("✓ Vision Transformer package imported successfully!")

✓ Vision Transformer package imported successfully!


In [ ]:
# Add the ViT package to Python path
# Update this path if your dataset has a different name
sys.path.append('/kaggle/input/vision-transformer-all-variants/src')

# Import ViT models
from vit_all_variants import (
    create_vit_s16, create_vit_s32,
    create_vit_b16, create_vit_b32,
    create_vit_l16, create_vit_l32,
    create_vit_h14, create_vit_h16,
    list_variants, compare_variants
)

print("✓ Vision Transformer package imported successfully!")

## Example 1: List All Available Models

In [9]:
# List all available variants with specifications
list_variants()


Available Vision Transformer Variants

VIT_S16
  Patch size: 16x16
  Layers: 12
  Embed dim: 384
  Attention heads: 6
  MLP dim: 1536
  Parameters: 21,665,664

VIT_S32
  Patch size: 32x32
  Layers: 12
  Embed dim: 384
  Attention heads: 6
  MLP dim: 1536
  Parameters: 22,493,952

VIT_B16
  Patch size: 16x16
  Layers: 12
  Embed dim: 768
  Attention heads: 12
  MLP dim: 3072
  Parameters: 85,798,656

VIT_B32
  Patch size: 32x32
  Layers: 12
  Embed dim: 768
  Attention heads: 12
  MLP dim: 3072
  Parameters: 87,455,232

VIT_L16
  Patch size: 16x16
  Layers: 24
  Embed dim: 1024
  Attention heads: 16
  MLP dim: 4096
  Parameters: 303,301,632

VIT_L32
  Patch size: 32x32
  Layers: 24
  Embed dim: 1024
  Attention heads: 16
  MLP dim: 4096
  Parameters: 305,510,400

VIT_H14
  Patch size: 14x14
  Layers: 32
  Embed dim: 1280
  Attention heads: 16
  MLP dim: 5120
  Parameters: 630,764,800

VIT_H16
  Patch size: 16x16
  Layers: 32
  Embed dim: 1280
  Attention heads: 16
  MLP dim: 5120
  Par

In [10]:
# Compare all variants side-by-side
compare_variants(image_size=224)


Variant      Patch    Layers   Embed    Heads    Params          Tokens    
VIT_S16      16       12       384      6          21,665,664   197       
VIT_S32      32       12       384      6          22,493,952   50        
VIT_B16      16       12       768      12         85,798,656   197       
VIT_B32      32       12       768      12         87,455,232   50        
VIT_L16      16       24       1024     16        303,301,632   197       
VIT_L32      32       24       1024     16        305,510,400   50        
VIT_H14      14       32       1280     16        630,764,800   257       
VIT_H16      16       32       1280     16        630,918,400   197       



## Example 2: Feature Extraction

Extract features from images for downstream tasks

In [12]:
# Create a ViT-B/16 model for feature extraction
model = create_vit_b16(
    image_size=224,
    include_top=False  # No classification head
)

print(f"Model created successfully!")
print(f"Total parameters: {model.count_params():,}")

Model created successfully!
Total parameters: 85,798,656


In [13]:
# Create dummy images (replace with your actual images)
batch_size = 4
images = tf.random.normal((batch_size, 224, 224, 3))

# Extract all tokens
all_features = model(images, training=False)
print(f"All tokens shape: {all_features.shape}")
print(f"  -> (batch_size={batch_size}, num_tokens=197, embed_dim=768)")

# Extract class token features only
cls_features = model.extract_features(images, training=False)
print(f"\nClass token features shape: {cls_features.shape}")
print(f"  -> (batch_size={batch_size}, embed_dim=768)")
print(f"\n✓ These 768-dimensional vectors can be used for:")
print("  • Image similarity search")
print("  • Transfer learning")
print("  • Clustering")
print("  • Downstream classification tasks")

All tokens shape: (4, 197, 768)
  -> (batch_size=4, num_tokens=197, embed_dim=768)

Class token features shape: (4, 768)
  -> (batch_size=4, embed_dim=768)

✓ These 768-dimensional vectors can be used for:
  • Image similarity search
  • Transfer learning
  • Clustering
  • Downstream classification tasks


## Example 3: Image Classification

Use ViT with a classification head

In [14]:
# Create model with classification head
num_classes = 10  # e.g., your dataset classes

classifier = create_vit_b32(  # Using B/32 for faster training
    image_size=224,
    include_top=True,
    num_classes=num_classes,
    dropout=0.1
)

print(f"Classifier created with {num_classes} output classes")
print(f"Total parameters: {classifier.count_params():,}")

Classifier created with 10 output classes
Total parameters: 87,462,922


In [15]:
# Compile the model
classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print("✓ Model compiled and ready for training!")

✓ Model compiled and ready for training!


In [16]:
# Test prediction (with dummy data)
test_images = tf.random.normal((4, 224, 224, 3))
logits = classifier(test_images, training=False)
probabilities = tf.nn.softmax(logits)

print(f"Logits shape: {logits.shape}")
print(f"Probabilities shape: {probabilities.shape}")
print(f"\nSample prediction for first image:")
print(f"  Predicted class: {tf.argmax(probabilities[0]).numpy()}")
print(f"  Confidence: {tf.reduce_max(probabilities[0]).numpy():.4f}")

Logits shape: (4, 10)
Probabilities shape: (4, 10)

Sample prediction for first image:
  Predicted class: 1
  Confidence: 0.6992


## Example 4: Transfer Learning with Frozen Backbone

Use ViT features with custom classification layers

In [17]:
# Create base model and freeze it
base_model = create_vit_b16(image_size=224, include_top=False)
base_model.trainable = False  # Freeze the backbone

print(f"Base model frozen: trainable={base_model.trainable}")
print(f"Base model parameters: {base_model.count_params():,}")

Base model frozen: trainable=False
Base model parameters: 85,798,656


In [18]:
# Build transfer learning model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = x[:, 0, :]  # Extract class token
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

transfer_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# Show parameter counts
total_params = transfer_model.count_params()
trainable_params = sum([tf.size(w).numpy() for w in transfer_model.trainable_weights])
non_trainable_params = sum([tf.size(w).numpy() for w in transfer_model.non_trainable_weights])

print(f"\nTransfer learning model created!")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Non-trainable parameters: {non_trainable_params:,}")
print(f"\n✓ Only the custom head will be trained, keeping ViT frozen!")



Transfer learning model created!
  Total parameters: 85,998,090
  Trainable parameters: 199,434
  Non-trainable parameters: 85,798,656

✓ Only the custom head will be trained, keeping ViT frozen!


## Example 5: Comparing Different Variants

Test different models for your use case

In [19]:
# Create different variants
variants = {
    'ViT-S/32': create_vit_s32,
    'ViT-S/16': create_vit_s16,
    'ViT-B/32': create_vit_b32,
    'ViT-B/16': create_vit_b16,
}

print("Model Comparison:")
print("=" * 80)
print(f"{'Model':<12} {'Parameters':<15} {'Output Shape':<25} {'Speed':<10}")
print("=" * 80)

test_image = tf.random.normal((1, 224, 224, 3))

for name, create_fn in variants.items():
    model = create_fn(image_size=224, include_top=False)
    output = model(test_image, training=False)
    params = model.count_params()
    
    # Simple speed indicator (smaller models are faster)
    if params < 30_000_000:
        speed = "⚡ Fast"
    elif params < 100_000_000:
        speed = "→ Medium"
    else:
        speed = "🐢 Slow"
    
    print(f"{name:<12} {params:>12,}   {str(tuple(output.shape)):<25} {speed:<10}")

print("=" * 80)

Model Comparison:
Model        Parameters      Output Shape              Speed     
ViT-S/32       22,493,952   (1, 50, 384)              ⚡ Fast    
ViT-S/16       21,665,664   (1, 197, 384)             ⚡ Fast    
ViT-B/32       87,455,232   (1, 50, 768)              → Medium  
ViT-B/16       85,798,656   (1, 197, 768)             → Medium  


## Example 6: Save and Load Models

In [22]:
# Create a model
model = create_vit_s16(image_size=224, include_top=False)

# Save weights
save_path = "vit_s16_weights.weights.h5"
model.save_weights(save_path)
print(f"✓ Model weights saved to: {save_path}")

# Load weights
new_model = create_vit_s16(
    image_size=224,
    include_top=False,
    weights=save_path
)
print(f"✓ Model weights loaded successfully!")

# Verify they produce same outputs
test_image = tf.random.normal((1, 224, 224, 3))
original_output = model(test_image, training=False)
loaded_output = new_model(test_image, training=False)
difference = tf.reduce_max(tf.abs(original_output - loaded_output)).numpy()

print(f"\nOutput difference: {difference:.10f} (should be ~0)")
print("✓ Models are identical!")

✓ Model weights saved to: vit_s16_weights.weights.h5
✓ Model weights loaded successfully!

Output difference: 0.0000000000 (should be ~0)
✓ Models are identical!


## Next Steps

Now you're ready to use Vision Transformers in your Kaggle projects!

**Recommendations:**
1. Start with **ViT-B/32** or **ViT-S/16** for quick experimentation
2. Use **ViT-B/16** for balanced accuracy and speed
3. Try **ViT-L/16** if you have GPU time and want maximum accuracy
4. Always use `include_top=False` for feature extraction
5. Freeze the backbone initially, then fine-tune if needed

**For More Examples:**
- Check the `examples/example_usage.py` file in this dataset
- Read the README.md for detailed documentation
- Experiment with different image sizes and configurations

Happy Kaggling! 🏆